In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import helper

data_dir = './brown.txt'

import numpy as np

def create_lookup_tables(text):
    vocab = set(text)
    vocab_to_int = {c: i for i, c in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    return (vocab_to_int, int_to_vocab)
    
def token_lookup():
    token_dict = {".":"<PERIOD>", ",": "<COMMA>", "\"":"<QOUTATION>", ";":"<>SEMICOLON", "!":"<EXCLAMATION>",
                 "?":"<QUESTION>", "(": "<LEFTPARANTHESES>", ")": "<RIGHTPARANTHESES>", "--":"<DASH>", "\n":"<RETURN>"}
    return token_dict

helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

from distutils.version import LooseVersion
import warnings
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import tensorflow as tf2
#import helper
import numpy as np

int_text, val_int_text, test_int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

def get_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learn_rate = tf.placeholder(tf.float32, name='learn_rate')
    return (inputs, targets, learn_rate)
    
def get_init_cell(batch_size, rnn_size):
    def build_cell(lstm_size):
        # Use a basic LSTM cell
        lstm = tf2.compat.v1.nn.rnn_cell.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf2.compat.v1.nn.rnn_cell.DropoutWrapper(lstm), output_keep_prob=keep_prob)
        return drop
    
    
    # Stack up multiple LSTM layers, for deep learning
    num_layers = 1
    cell = tf2.compat.v1.nn.rnn_cell.MultiRNNCell([build_cell(rnn_size) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, dtype=tf.float32)
    initial_state = tf.identity(initial_state, name="initial_state")
    return (cell, initial_state)
    
def get_embed(input_data, vocab_size, embed_dim):
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed
    
def build_rnn(cell, inputs):
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)#,initial_state=initial_state)
    final_state = tf.identity(final_state, name="final_state")
    return outputs, final_state
    
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed)
    
    logits = tf2.compat.v1.layers.dense(outputs, vocab_size, activation=None)
    return logits, final_state
    
def get_batches(int_text, batch_size, seq_length):
    words_per_batch = batch_size * seq_length
    num_batch = len(int_text)//words_per_batch
    
    input_text = np.array(int_text[:(num_batch*words_per_batch)])
    target_text = np.array(int_text[1:(num_batch * words_per_batch) + 1])

    target_text[-1] = input_text[0]
    
    x_batches = np.split(input_text.reshape(batch_size, -1), num_batch, 1)
    y_batches = np.split(target_text.reshape(batch_size, -1), num_batch, 1)
    
    batches = np.array(list(zip(x_batches, y_batches)))
    return batches
    
# Number of Epochs
num_epochs = 50
# Batch Size
batch_size = 100
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 10
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 50

save_dir = '/content/drive/MyDrive/nlp_models/model_2/save'

!pip install tensorflow_addons
import tensorflow_addons as tfa


train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = tfa.seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)
    
batches = get_batches(int_text, batch_size, seq_length)
#train_graph = tf.Graph()
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')
    
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

In [ ]:
import tensorflow as tf
import numpy as np
import helper

from google.colab import drive
drive.mount('/content/drive')

def token_lookup():
    token_dict = {".":"<PERIOD>", ",": "<COMMA>", "\"":"<QOUTATION>", ";":"<>SEMICOLON", "!":"<EXCLAMATION>",
                 "?":"<QUESTION>", "(": "<LEFTPARANTHESES>", ")": "<RIGHTPARANTHESES>", "--":"<DASH>", "\n":"<RETURN>"}
    return token_dict

_,_,_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = 15, "/content/drive/MyDrive/nlp_models/model_1/save"
def get_tensors(loaded_graph):
    input_tensor = loaded_graph.get_tensor_by_name("input:0")
    initial_state_tensor = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state_tensor = loaded_graph.get_tensor_by_name("final_state:0")
    prob_tensor = loaded_graph.get_tensor_by_name("probs:0")

    return input_tensor, initial_state_tensor, final_state_tensor, prob_tensor
    
#test_text = [int_to_vocab[word] for word in test_int_text]
#test_text = " ".join(test_text)
#test_sent_text = test_text.split("<period>")

test_sentences = input("Enter sentence:")
token_dict = token_lookup()
for key, token in token_dict.items():
	test_sentences = test_sentences.replace(key, ' {} '.format(token))
test_sentences = test_sentences.lower()
test_sentences = test_sentences.split("<period>")

all_perp = []

loaded_graph = tf.Graph()
with tf.compat.v1.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.compat.v1.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)
    #print(final_state)
    # Sentences generation setup
    for line in test_sentences:
        if len(line.split()) == 0:
            continue
        gen_length = len(line.split())-2
        prime_word = line.split()[0]#'moe_szyslak'
        i=1
        prob = 1.0
        gen_sentences = [prime_word]
            
        prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

        # Generate sentences
        for n in range(gen_length):
            # Dynamic Input
            dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
            dyn_seq_length = len(dyn_input[0])
          
            # Get Prediction
            probabilities, prev_state = sess.run(
                [probs, final_state],
                {input_text: dyn_input, initial_state: prev_state})

            #pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)
            prob *= probabilities[0][dyn_seq_length-1][vocab_to_int[line.split()[i]]]
            i+=1
            pred_word = line.split()[i]
            gen_sentences.append(pred_word)
        if prob==0.0 or gen_length+1==0:
            continue
        # Remove tokens
        gen_sen = ' '.join(gen_sentences)
        for key, token in token_dict.items():
            #ending = ' ' if key in ['\n', '(', '"'] else ''
            gen_sen = gen_sen.replace(' ' + token.lower(), key)
        gen_sen = gen_sen.replace('\n ', ' ')
        gen_sen = gen_sen.replace('( ', '(')

        print(gen_sen)
        #the_file.write(tv_script+'\t')
        perp = helper.compute_perplexity(prob, gen_length+1)
        print(gen_sen)
        print("Perplexity:", perp)
            #the_file.write(str(perp))
            #the_file.write("\n")       
        all_perp.append(perp)
    #the_file.write(str(sum(all_perp) / len(all_perp)))